# Exploring trading pairs

An example how to filter and plot candles for pair data

We will

* Download pair and exchange map ("the universe")

* Filter out SushiSwap pairs

* Download 4 hour candle data for all pairs

* Display top 10 pairs by their 30d volume, nominated in the USD

First let's import libraries and initialise our dataset client.

In [11]:

import pandas as pd

from capitalgram.chain import ChainId
from capitalgram.pair import PairUniverse

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Get the map of exchanges and pairs we are working on

In [12]:
# Decompress the pair dataset to Python map
columnar_pair_table = capitalgram.fetch_pair_universe()

# Exchange map data is so small it does not need any decompression
exchange_universe = capitalgram.fetch_exchange_universe()

print(f"Total pairs {len(columnar_pair_table)}, total exchanges {len(exchange_universe.exchanges)}")

Total pairs 47686, total exchanges 231


## Filtering pairs trading on Sushiswap

We are only interested in SushiSwap, the mosh pit of a degens, so filter pairs that are on this exchange only.
We specifically pick Sushiswap on Ethereum mainnet,
as it is the oldest and most active for all Sushiswap instances across EVM blockchains like Polygon, BSC, Fantom, etc.

Also we do not yet have multichain data on the dataset server, when I am writing this tutorial so...

In [13]:
# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

print(f"Sushiswap on Ethereum has {len(sushi_pairs)} pairs")

Sushiswap on Ethereum has 1308 pairs


## Calculating combined buy and sell volume

Now let's get pairs sorted out by 30d buy and sell volume

In [14]:
# Avoid doing set operations on a Pandas read-only view
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
sushi_pairs = sushi_pairs.copy()
total_volume_30d = sushi_pairs["buy_volume_30d"] + sushi_pairs["sell_volume_30d"]
sushi_pairs["total_30d_volume"] = total_volume_30d
# Sort by highest volume firs   t
sushi_pairs = sushi_pairs.sort_values("total_30d_volume", ascending=False)

print("total_30d_volume column sum added")

total_30d_volume column added


## Output table for top-10 pairs

Then let's see the top 10 pairs we got and choose colums to display.

In [15]:
# See Pandas official documentation for table visualisation
# https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

output = sushi_pairs[["base_token_symbol", "quote_token_symbol", "total_30d_volume"]].copy()

output.style.format({
  'total_30d_volume': lambda val: f'${val:,.2f}',
})

output = output.rename(columns={
    'base_token_symbol': 'Base token',
    'quote_token_symbol': 'Quote token',
    'total_30d_volume': 'Volume 30d USD'})

top_10 = output.head(10)

top_10.style.format({
  'Volume 30d USD': lambda val: f'${val:,.2f}',
}).hide_index()

Base token,Quote token,Volume 30d USD
WETH,USDC,"$1,089,837,312.00"
WETH,USDT,"$684,412,288.00"
WETH,DAI,"$548,666,752.00"
SUSHI,WETH,"$395,642,400.00"
WETH,WBTC,"$308,113,696.00"
OHM,DAI,"$147,783,568.00"
AAVE,WETH,"$142,700,480.00"
LINK,WETH,"$92,933,976.00"
YFI,WETH,"$91,865,696.00"
CRV,WETH,"$89,533,808.00"
